In [ ]:
%matplotlib widget

import numpy as np
import h5py
import ipywidgets as w
from pytransform3d.plot_utils import make_3d_axis
from pytransform3d.transformations import plot_transform, transform_from
from pytransform3d.rotations import matrix_from_quaternion

def reorder_quat(q4):
    """将四元数重排到 wxyz（pytransform3d 默认）"""
    q4 = np.asarray(q4, dtype=float)
    x, y, z, w = q4
    return np.array([w, x, y, z], dtype=float)

def normalize_quat(q4):
    n = np.linalg.norm(q4)
    return q4 if n == 0 else q4 / n

# --- HDF5 数据读取 ---
def load_right_hand(path, pose_key="action/rightHand/pose", quat_key="action/rightHand/quaternion"):
    with h5py.File(path, "r") as f:
        P = np.asarray(f[pose_key])
        Q = np.asarray(f[quat_key])
    pos = P[:, :3].astype(float)
    quat = Q[:, :4].astype(float)
    return pos, quat

HDF5_PATH = "/home/ls/Desktop/tf_test/base_action_data3/Data/Data.hdf5"
pos_lh, quat_lh = load_right_hand(HDF5_PATH)
N = len(pos_lh)

# --- LH->RH 转换 ---
def lh_to_rh_pos(p):
    # xyz -> xy -z
    return np.array([p[0], p[1], -p[2]])

def lh_to_rh_quat(q):
    # xyzw -> xyz -w
    return np.array([q[0], q[1], -q[2], -q[3]])

pos_rh = np.array([lh_to_rh_pos(p) for p in pos_lh])
quat_rh = np.array([lh_to_rh_quat(q) for q in quat_lh])

extent = float(np.max(np.abs(pos_rh))) if pos_rh.size else 1.0
extent = max(0.5, min(extent * 1.5, 5.0))

# --- 控件 ---
frame = w.IntSlider(0, min=0, max=N-1, step=1, description='frame')
play  = w.Play(interval=40, value=0, min=0, max=N-1, step=1)
w.jslink((play, 'value'), (frame, 'value'))
axis_len = w.FloatSlider(0.2, min=0.05, max=0.6, step=0.01, description='axis_len')
show_world = w.Checkbox(value=True, description='show world')

# --- 绘制 ---
ax = make_3d_axis(ax_s=1.0)
ax.set_title('RightHand (converted LH -> RH)')
ax.set_xlim(-extent, extent); ax.set_ylim(-extent, extent); ax.set_zlim(-extent, extent)
ax.set_box_aspect([1,1,1])

def draw(_=None):
    ax.cla()
    ax.set_title('RightHand (converted LH -> RH)')
    ax.set_xlim(-extent, extent); ax.set_ylim(-extent, extent); ax.set_zlim(-extent, extent)
    ax.set_box_aspect([1,1,1])

    i = int(frame.value)
    t = pos_rh[i]
    q = quat_rh[i]  # 已经是 RH 的四元数 (xyzw)
    q = reorder_quat(q)
    q = normalize_quat(q)
    R = matrix_from_quaternion(q)
    T = transform_from(R, t)


    if show_world.value:
        plot_transform(ax, A2B=np.eye(4), s=0.25)  # 世界坐标 (RH)
    plot_transform(ax, A2B=T, s=float(axis_len.value))

    ax.figure.canvas.draw_idle()

for wd in [frame, axis_len, show_world]:
    wd.observe(draw, names='value')

ui = w.VBox([
    w.HBox([frame, play]),
    w.HBox([axis_len, show_world])
])
display(w.VBox([ui, ax.figure.canvas]))
draw()
# 红色x 绿色y 蓝色z